In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir())

# Any results you write to the current directory are saved as output.

['utils.py', 'generate_missing_data.ipynb', 'test.csv', 'dataset', 'generate_dataset.py', 'model', 'generate_missing_data.py', 'GAN.ipynb', '.ipynb_checkpoints', 'fill_dataset.ipynb', 'MLP_Classifier.ipynb', 'LICENSE', 'generate_dataset.ipynb', 'generate_ones.py', 'train.csv', '.git', 'README.md', '.gitignore', 'generate csv files.ipynb']


In [4]:
from sklearn.utils import shuffle 

df = pd.read_csv("dataset/train.csv", index_col=0)
df = shuffle(df)
df.sample(2)

,target,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
210,4,67,69,75,56,52,47,56,78,63,...,52,52,68,60,79,56,81,48,55,32
199,3,113,96,97,89,88,108,102,105,102,...,89,89,97,92,89,89,91,109,83,107


In [5]:
y = df.iloc[:,:1]
x = df.iloc[:,1:]

# Input data

In [6]:
from math import ceil

n = x.shape[0]
n_train = ceil(0.8 * n)
n_test = ceil(0.2 * n)

x_train = x[:n_train]
y_train = y[:n_train]

x_test = x[n_train:]
y_test = y[n_train:]

y_train = np.squeeze(y_train)
y_test = np.squeeze(y_test)

# Building the model and evaluating

In [7]:
from sklearn.neural_network import MLPClassifier

mlp_model = MLPClassifier(solver='adam', 
                          alpha=1e-5, 
                          hidden_layer_sizes=(1024, 128), 
                          random_state=1, 
                          max_iter=150)

print(mlp_model)

mlp_model.fit(x_train, y_train)                         
pred_mlp =  mlp_model.predict(x_test)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1024, 128), learning_rate='constant',
       learning_rate_init=0.001, max_iter=150, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='adam', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)


In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn import metrics

print("\n\nTesting using MLP")
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, pred_mlp))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, pred_mlp))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred_mlp))) 



Testing using MLP
Mean Absolute Error: 0.37
Mean Squared Error: 2.01
Root Mean Squared Error: 1.4177446878757824


In [9]:
print("\n\nMLP")
print("--- confusion_matrix ---")
print(confusion_matrix(y_test, pred_mlp))  
print("\n--- classification report ---")
print(classification_report(y_test, pred_mlp))  
print("\nmodel accuracy: ", accuracy_score(y_test, pred_mlp))



MLP
--- confusion_matrix ---
[[ 5  0  0  0  0  0  0  0  0  0]
 [ 0 14  0  0  0  0  0  0  0  0]
 [ 0  0 12  0  0  0  0  0  1  0]
 [ 0  0  0  6  0  2  0  0  0  0]
 [ 0  0  0  0 10  0  0  0  0  0]
 [ 0  0  0  0  0 11  0  0  0  0]
 [ 1  0  1  0  1  0 11  0  0  0]
 [ 0  0  0  0  0  0  0  7  0  0]
 [ 1  0  0  0  0  0  0  0  3  1]
 [ 0  0  0  1  0  0  0  0  0 12]]

--- classification report ---
             precision    recall  f1-score   support

          0       0.71      1.00      0.83         5
          1       1.00      1.00      1.00        14
          2       0.92      0.92      0.92        13
          3       0.86      0.75      0.80         8
          4       0.91      1.00      0.95        10
          5       0.85      1.00      0.92        11
          6       1.00      0.79      0.88        14
          7       1.00      1.00      1.00         7
          8       0.75      0.60      0.67         5
          9       0.92      0.92      0.92        13

avg / total       0.92

# Recognizing numbers (or not)

In [13]:
df_test = pd.read_csv("dataset/test.csv", index_col=0)
df_test = shuffle(df_test)
df_test.sample(2)

,target,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
28,2,109,93,99,86,100,102,105,91,92,...,97,96,97,99,89,87,91,100,87,97
66,6,108,89,95,105,89,99,100,111,115,...,104,95,97,91,105,96,88,97,88,109


In [25]:
test = df_test.iloc[:, 1:].values
test_targets = df_test.iloc[:, 0].values
test 

array([[108, 128, 108, ..., 131,  99, 100],
       [ 69,  59,  80, ...,  83,  87,  80],
       [ 62,  54,  69, ...,  44,  72,  68],
       ...,
       [117, 109, 113, ..., 117,  99, 121],
       [110, 111, 113, ..., 116, 119, 120],
       [ 71,  78,  76, ...,  80,  71,  56]])

In [17]:
preds =  mlp_model.predict(test)

In [26]:
print("\nmodel accuracy: ", accuracy_score(test_targets, preds))


model accuracy:  0.8383838383838383


# Testing with real numbers of dataset

In [29]:
df_real = pd.read_csv("dataset/mnist.csv")
df_real = shuffle(df)
df_real.sample(2)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
20405,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33438,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
data_real = df_real.iloc[:,1:].values
targets_real = df_real.iloc[:,0].values
preds =  mlp_model.predict(data_real)

In [32]:
print("\nmodel accuracy: ", accuracy_score(targets_real, preds))


model accuracy:  0.44926190476190475


# Results are not ok, but gans are fun